<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pip
import sys
#if not 'sklearn' in sys.modules.keys():
#    pip.main(['install', 'sklearn'])
#if not 'kaggle' in sys.modules.keys():
#    pip.main(['install', 'kaggle'])
import random

print("Random number with seed 2020")
# first call
random.seed(2020)

Random number with seed 2020


In [ ]:
import numpy as np
import pandas as pd
import graphviz

from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

# Read the data from GitHub
train = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')


In [ ]:
#Le classi sono 1=Good 0=Disappointing
train["Quality"] = np.where(train["Quality"].str.contains("Good"), 1, 0)

In [ ]:
# pull data into target (y) and predictors (X)
#La y è la classe, cioè la Quality
train_y = train.Quality
#seleziono colonne di interesse, non metto l'ID perchè non mi interessa
predictor_cols = ['fixed.acidity','volatile.acidity','citric.acid','residual.sugar','chlorides','free.sulfur.dioxide','total.sulfur.dioxide','density','pH','sulphates','alcohol']

# La x sono gli attributi
train_X = train[predictor_cols]

# Sostituisce i missing values con la media e lo applica alle x
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(train_X)

# Impute our data, then train
train_X_imp = imp.transform(train_X)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X_imp = scaler.fit_transform(train_X_imp)

In [ ]:
#Uso l'80% dei dati per train e il restante 20% per test
xTrain, xTest, yTrain, yTest = train_test_split(train_X_imp, train_y, train_size = 0.8, random_state = 0)

In [ ]:
def generateSubmission(myModel, submissionFile, description):
    # Read the test data
    test = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/test.csv')


    # Treat the test data in the same way as training data. In this case, pull same columns.
    test_X = test[predictor_cols]

    # Impute each test item, then predict
    test_X_imp = imp.transform(test_X)
    
    # Use the model to make predictions
    predicted_q = myModel.predict(test_X_imp)
    # We will look at the predicted Qualities to ensure we have something sensible.
    print(predicted_q)
    
    #submission file
    my_submission = pd.DataFrame({'Id': test.Id, 'Quality': predicted_q})
    # you could use any filename. We choose submission here
    my_submission.to_csv(submissionFile, index=False)
    
    #Submit authomatically; kaggle API authentication needed
    #!kaggle competitions submit -c mldm-classification-competition-2020 -f {submissionFile} -m '{description}'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
#DA CAMBIARE! METODO E PARAMETRI
svc = SVC()
#Range di parametri che vengono esplorati per trovare la combinazione migliore
parameter_grid = {'kernel': ['poly'],
                  'degree': [3,4,5],
                  'C': [2,3,5],
                  #'max_iter' : [200,500]
                  }


cross_validation = StratifiedKFold(n_splits=10)
cross_validation.get_n_splits(train_X_imp, train_y)
#Create the scoring dictionary
SCORING = {'accuracy': 'accuracy',
'balanced_accuracy': 'balanced_accuracy',
'precision': 'precision_macro',
'recall': 'recall_macro',
'f1': 'f1_macro'}

#grid_search = GridSearchCV(dtc, param_grid=parameter_grid, cv=cross_validation)
grid_search = GridSearchCV(svc, param_grid=parameter_grid, cv=cross_validation, scoring=SCORING,return_train_score=True, refit='f1')

grid_search.fit(train_X_imp, train_y)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [2, 5, 10], 'degree': [3, 4, 5],
                         'kernel': ['poly']},
             pre_dispatch='2*n_jobs', refit='f1', return_train_score=True,
             scoring={'accuracy': 'accuracy',
                      'balanced_accuracy': 'balanced_accuracy',
                      'f1': 'f1_macro', 'precision': 'precision_macro',
                      'recall': 'recall_macro'},
             verbose=0)

In [ ]:
grid_search.cv_results_

In [ ]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_dtc = grid_search.best_estimator_
best_dtc

Best score: 0.663674637203174
Best parameters: {'C': 10, 'degree': 4, 'kernel': 'poly'}


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=4, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
my_model=best_dtc
my_model.fit(train_X_imp, train_y)
my_model.score(train_X_imp, train_y)

0.8191458870736601

In [ ]:
y_pred = my_model.predict(train_X_imp)
print("Confusion Matrix:")
print(confusion_matrix(train_y, y_pred))

print("Classification Report:")
print(classification_report(train_y, y_pred))

Confusion Matrix:
[[ 673  532]
 [  99 2185]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.56      0.68      1205
           1       0.80      0.96      0.87      2284

    accuracy                           0.82      3489
   macro avg       0.84      0.76      0.78      3489
weighted avg       0.83      0.82      0.81      3489



In [ ]:
# generate a submission file
generateSubmission(my_model,'SVM_poly2.0.csv', "User defined SVM evaluated with Grid Search and poly kernel")

[1 0 0 ... 1 0 0]
